<a href="https://colab.research.google.com/github/ejrbdx/Advance-Research-Methods-2024/blob/main/Homework_2-current%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd '/content/drive/MyDrive/Advanced Research Methods/Homework 2'

/content/drive/MyDrive/Advanced Research Methods/Homework 2


In [2]:
import pandas as pd
import random, math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import fftpack
import torch.utils.data as utils
import os
from pathlib import Path
from sklearn.metrics import mean_absolute_percentage_error
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [3]:
def reshape(root):

 tps_df = pd.read_pickle(root)

 reshaped_tps_df = pd.DataFrame()
 reshaped_tps_df['TIME'] = tps_df.time.unique()
 for seg in tps_df.segmentID.unique():
     column = tps_df[tps_df['segmentID'] == seg][['time','TrafficIndex_GP']].drop_duplicates(subset=['time'])
     column.columns = ['TIME', str(seg)]
     reshaped_tps_df = reshaped_tps_df.join(column.set_index('TIME'), on='TIME')

 reshaped_tps_df = reshaped_tps_df.set_index('TIME')
 return reshaped_tps_df



In [4]:
train_data = reshape('tps_df.pkl')


In [95]:
# root = Path('Test')
# file = []
# for fldr in root.iterdir():
#   file = fldr.name
#   print()




In [ ]:
# inputs = []
# targets = []

# for segid in df.columns:
#   TI_series = df[segid].values
#   print(segid)
#   print(TI_series)

#   # for t in range(0,len(TI_series)-(36+12)):
#   #     x = TI_series[t:t+36]
#   #     y = TI_series[t+36:(t+36+12)]

#   #     inputs.append(x)
#   #     targets.append(y)





In [5]:
class TrafficForecast(Dataset):
    def __init__(self,df, window, horizon):
        self.window = window
        self.horizon = horizon
        self.df = df
        self.inputs = []
        self.targets = []

        df = pd.read_pickle(self.pkl_path)
        df['time'] = pd.to_datetime(df['time'])
        df['unix_timestamp'] = df['time'].astype(int) / 10**9
        df['dow'] = df['time'].dt.weekday
        df['hour'] = df['time'].dt.hour
        # df['min'] = df['time'].dt.minute
        self.df = df.sample(frac=1).reset_index(drop=True)
        self.seg_ids = self.df['segmentID'].unique()

        self.setup_forecast()

    def setup_forecast(self):
        for segid in self.df.columns:
          self.df = self.df.fillna(0)
          TI_series = self.df[segid].values

          for t in range(0,len(TI_series)-(self.window+self.horizon)):
              x = TI_series[t:t+self.window]
              y = TI_series[t+self.window:(t+self.window+self.horizon)]


              self.inputs.append(x)
              self.targets.append(y)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,idx):
        X = torch.tensor(self.inputs[idx],dtype=torch.float32).reshape(self.window,1)
        y = torch.tensor(self.targets[idx],dtype=torch.float32)

        return {'inputs':X,'outputs':y}


In [6]:
tobj = TrafficForecast(train_data,36,12)


In [7]:
bs = 200
dataloader_train = DataLoader(tobj, batch_size=bs, shuffle=False, num_workers=4,drop_last=True)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
from models import MyLSTM, MyGRU, MyRNNCell, MyRNN

In [9]:
model = MyLSTM(n_inputs=1, n_hidden=10, n_rnnlayers=2, n_outputs=12)
# model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
mean_loss = []
losses = []

n_epochs = 1

for it in tqdm(range(n_epochs)):
  # zero the parameter gradients
  for i_batch, sample_batched in enumerate(dataloader_train):
    optimizer.zero_grad()
    outputs = model(sample_batched['inputs'])
    loss = criterion(outputs, sample_batched['outputs'])
    losses.append(loss.item())
    loss.backward()
    optimizer.step()



  mean_loss.append(np.mean(losses))

  if (it+1) % 1 == 0:
    print(f'Epoch {it+1}/{n_epochs}, Training Loss: {np.mean(losses):.4f}')

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 1/1 [03:22<00:00, 202.69s/it]

Epoch 1/1, Training Loss: 0.0065


In [12]:
t6 = reshape('Test/tps_6.pkl')

# t9 = reshape('Test/tps_9.pkl')
# t2 = reshape('Test/tps_2.pkl')
# t14 = reshape('Test/tps_14.pkl')
# t4 = reshape('Test/tps_4.pkl')
# t5 = reshape('Test/tps_5.pkl')
# t3 = reshape('Test/tps_3.pkl')
# t12 = reshape('Test/tps_12.pkl')
# t7 = reshape('Test/tps_7.pkl')
# t13 = reshape('Test/tps_13.pkl')
# t10 = reshape('Test/tps_10.pkl')
# t1 = reshape('Test/tps_1.pkl')
# t11 = reshape('Test/tps_11.pkl')
# t15 = reshape('Test/tps_15.pkl')
# t8 = reshape('Test/tps_8.pkl')

In [79]:
len(t6.columns)

87

In [75]:
class Test(Dataset):
    def __init__(self,df):

        self.df = df
        self.inputs = []

        self.setup_forecast()

    def setup_forecast(self):
        for segid in self.df.columns:
          self.df = self.df.fillna(0)
          TI_series = self.df[segid].values

          for t in range(0,len(TI_series)):
              x = TI_series[t]

              self.inputs.append(x)
        print(self.inputs)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,idx):
        X = torch.tensor(self.inputs[idx],dtype=torch.float32).reshape(1,1)


        return {'inputs':X}

In [76]:
tobj6 = Test(t6)
# tobj9 = TrafficForecast(t9,36,12)
# tobj2 = TrafficForecast(t2,36,12)
# tobj14 = TrafficForecast(t14,36,12)
# tobj4 = TrafficForecast(t4,36,12)
# tobj5 = TrafficForecast(t5,36,12)
# tobj3 = TrafficForecast(t3,36,12)
# tobj12 = TrafficForecast(t12,36,12)
# tobj7 = TrafficForecast(t7,36,12)
# tobj13 = TrafficForecast(t13,36,12)
# tobj10 = TrafficForecast(t10,36,12)
# tobj1 = TrafficForecast(t1,36,12)
# tobj11 = TrafficForecast(t11,36,12)
# tobj15 = TrafficForecast(t15,36,12)
# tobj8 = TrafficForecast(t8,36,12)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.997, 0.999, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.985, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.992, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.998, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.997, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.993, 1.0, 1.0, 1.0, 1.0, 1.0, 0.999, 0.998, 1.0, 0.999, 0.997, 0.995, 0.996, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

In [107]:
dataloader6 = DataLoader(tobj6, batch_size=36, shuffle=False, num_workers=4,drop_last=True)
# dataloader9 = DataLoader(tobj9, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader2 = DataLoader(tobj2, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader14 = DataLoader(tobj14, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader4 = DataLoader(tobj4, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader5 = DataLoader(tobj5, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader3 = DataLoader(tobj3, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader12 = DataLoader(tobj12, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader7 = DataLoader(tobj7, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader13 = DataLoader(tobj13, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader10 = DataLoader(tobj10, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader1 = DataLoader(tobj1, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader11 = DataLoader(tobj11, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader15 = DataLoader(tobj15, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
# dataloader8 = DataLoader(tobj8, batch_size=1, shuffle=False, num_workers=4,drop_last=True)

In [115]:
def evaluation(model,dataloader_test, bs):
    # rmse = []
    # mapes = []
    preds_re = []

    for i_test,sample_test in enumerate(dataloader_test):

        preds = model(sample_test['inputs'])

        # targets = sample_test['outputs']
        # error = ((preds-targets)**2).sum(1).mean().detach().numpy()
        # mape = ((preds-targets)*100).abs().sum(1).mean().detach().numpy()
        # rmse.append(error)
        # mapes.append(mape)
        preds_re.append(preds.detach().numpy())
    # return {'rmse':np.mean(rmse),'mae':np.mean(mapes)}
    return preds_re

In [117]:
results = evaluation(model,dataloader6,1)
print(results)

[array([[0.81055266, 0.6077154 , 0.65779483, 0.6532784 , 0.62370163,
        0.8017217 , 0.7654983 , 0.8073579 , 0.65348697, 0.64709735,
        0.72681105, 0.3583127 ],
       [0.81055266, 0.6077154 , 0.65779483, 0.6532784 , 0.62370163,
        0.8017217 , 0.7654983 , 0.8073579 , 0.65348697, 0.64709735,
        0.72681105, 0.3583127 ],
       [0.81055266, 0.6077154 , 0.65779483, 0.6532784 , 0.62370163,
        0.8017217 , 0.7654983 , 0.8073579 , 0.65348697, 0.64709735,
        0.72681105, 0.3583127 ],
       [0.81055266, 0.6077154 , 0.65779483, 0.6532784 , 0.62370163,
        0.8017217 , 0.7654983 , 0.8073579 , 0.65348697, 0.64709735,
        0.72681105, 0.3583127 ],
       [0.81055266, 0.6077154 , 0.65779483, 0.6532784 , 0.62370163,
        0.8017217 , 0.7654983 , 0.8073579 , 0.65348697, 0.64709735,
        0.72681105, 0.3583127 ],
       [0.81055266, 0.6077154 , 0.65779483, 0.6532784 , 0.62370163,
        0.8017217 , 0.7654983 , 0.8073579 , 0.65348697, 0.64709735,
        0.72681105

In [121]:
output_test = pd.DataFrame(results)

ValueError: Must pass 2-d input. shape=(87, 36, 12)

In [14]:
# mean_loss = []
# losses = []
# test_losses = []
# mean_test_loss = []
# n_epochs = 3

# for it in tqdm(range(n_epochs)):
#   # zero the parameter gradients
#   for i_batch, sample_batched in enumerate(dataloader_train):
#     optimizer.zero_grad()
#     outputs = model(sample_batched['inputs'])
#     loss = criterion(outputs, sample_batched['outputs'])
#     losses.append(loss.item())
#     loss.backward()
#     optimizer.step()

#   for i_test,sample_test in enumerate(dataloader_test):

#       outputs = model(sample_test['inputs'])
#       test_loss = criterion(outputs,sample_test['outputs'])
#       test_losses.append(test_loss.item())

#   mean_loss.append(np.mean(losses))
#   mean_test_loss.append(np.mean(test_losses))
#   if (it+1) % 1 == 0:
#     print(f'Epoch {it+1}/{n_epochs}, Training Loss: {np.mean(losses):.4f}, Testing Loss: {np.mean(test_losses):.4f}')

  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
 33%|███▎      | 1/3 [03:03<06:07, 183.54s/it]

Epoch 1/3, Training Loss: 0.0183, Testing Loss: 0.0038


 67%|██████▋   | 2/3 [05:58<02:58, 178.68s/it]

Epoch 2/3, Training Loss: 0.0121, Testing Loss: 0.0038


100%|██████████| 3/3 [08:56<00:00, 178.98s/it]

Epoch 3/3, Training Loss: 0.0100, Testing Loss: 0.0037
